In [2]:
import numpy as np

# Example: A simple 3x3 grayscale image (values range from 0 to 255)
original_image = np.array([
    [120, 135, 140],
    [150, 160, 170],
    [180, 190, 200]
], dtype=np.float32)

# Normalize the image to range [0, 1] for simplicity
x_0 = original_image / 255.0

x_0

array([[0.47058824, 0.5294118 , 0.54901963],
       [0.5882353 , 0.627451  , 0.6666667 ],
       [0.7058824 , 0.74509805, 0.78431374]], dtype=float32)

In [3]:
# Parameters for the diffusion process
T = 5  # Total diffusion steps
beta = np.linspace(0.01, 0.05, T)  # Noise schedule
alpha = 1 - beta  # Alpha values
alpha_bar = np.cumprod(alpha)  # Cumulative product of alpha

In [6]:
beta, alpha, alpha_bar

(array([0.01, 0.02, 0.03, 0.04, 0.05]),
 array([0.99, 0.98, 0.97, 0.96, 0.95]),
 array([0.99      , 0.9702    , 0.941094  , 0.90345024, 0.85827773]))

In [9]:
# Simulate the forward process to add noise step by step
noisy_images = [x_0]  # Store noisy images at each step
for t in range(T):
    noise = np.random.normal(0, 1, x_0.shape)  # Gaussian noise
    x_t = np.sqrt(alpha_bar[t]) * x_0 + np.sqrt(1 - alpha_bar[t]) * noise
    noisy_images.append(x_t)

print(noisy_images)

[array([[0.47058824, 0.5294118 , 0.54901963],
       [0.5882353 , 0.627451  , 0.6666667 ],
       [0.7058824 , 0.74509805, 0.78431374]], dtype=float32), array([[0.68296214, 0.56073273, 0.50713195],
       [0.58518218, 0.6602204 , 0.66971037],
       [0.60718026, 0.66761336, 0.77230533]]), array([[0.37861301, 0.64234659, 0.80141181],
       [0.51872403, 0.36642085, 0.54318953],
       [0.72752115, 0.80473619, 0.67067174]]), array([[0.64997351, 0.21230692, 0.2438886 ],
       [0.42027929, 0.61388054, 1.11743167],
       [0.98093237, 0.83551109, 0.07229484]]), array([[0.3121935 , 0.40092992, 0.57592763],
       [0.52434871, 0.38306657, 0.77995134],
       [0.32587409, 1.12719767, 0.45698493]]), array([[1.14061477, 0.64394941, 0.83882275],
       [1.1755926 , 0.96745499, 0.69434355],
       [0.56438157, 0.56065071, 0.74635056]])]


In [16]:
# Choose one noisy image (e.g., from step t=3) for reverse process demonstration
t = 4
x_t = noisy_images[t]
predicted_noise = np.random.normal(0, 1, x_t.shape)  # Simulated predicted noise - Trained NN, often U-Net

# Reverse process: Remove noise step by step
sigma_t = np.sqrt(beta[t])  # Small noise factor for diversity
x_t_minus_1 = (
    (1 / np.sqrt(alpha[t])) * (x_t - (1 - alpha[t]) / np.sqrt(1 - alpha_bar[t]) * predicted_noise)
    + sigma_t * np.random.normal(0, 1, x_t.shape)
)

# Display the results
original_image, noisy_images[t], x_t_minus_1 * 255  # Reverse step scaled back to original range

(array([[120., 135., 140.],
        [150., 160., 170.],
        [180., 190., 200.]], dtype=float32),
 array([[0.3121935 , 0.40092992, 0.57592763],
        [0.52434871, 0.38306657, 0.77995134],
        [0.32587409, 1.12719767, 0.45698493]]),
 array([[156.36442002,  62.37752183, 117.55792342],
        [122.08466102, 148.22025955, 244.40168165],
        [155.68335883, 226.20426891,  42.77584885]]))

In [20]:
import numpy as np

# Simulate a neural network's weight matrix (3x3 for simplicity)
weights = np.random.randn(3, 3)  # Initial weights

# Learning rate for gradient descent
learning_rate = 0.01

# Simulate the true image (normalized 3x3 image)
x_0 = np.array([
    [0.47, 0.53, 0.55],
    [0.59, 0.63, 0.67],
    [0.71, 0.75, 0.78]
])

# Simulate noise added to the image in forward diffusion
true_noise = np.random.normal(0, 1, x_0.shape)  # True noise

# Simulate the noisy image at time step t
t = 3
alpha_bar_t = 0.5  # Cumulative product of alpha at t
x_t = np.sqrt(alpha_bar_t) * x_0 + np.sqrt(1 - alpha_bar_t) * true_noise

# Simulate the neural network's predicted noise (epsilon_theta)
def predict_noise(weights, x_t):
    return np.dot(x_t, weights)  # Simplified linear model for noise prediction - This is the trained NN in practice

predicted_noise = predict_noise(weights, x_t)

# Calculate the loss (MSE)
loss = np.mean((true_noise - predicted_noise) ** 2)
print(f"Loss before update: {loss:.4f}")

# Calculate gradients of the loss with respect to weights
# Gradient = (2/N) * (predicted_noise - true_noise) * x_t
gradients = (2 / x_t.size) * np.dot(x_t.T, (predicted_noise - true_noise))

# Update weights using gradient descent
weights -= learning_rate * gradients

# Predict noise with updated weights
predicted_noise_updated = predict_noise(weights, x_t)

# Calculate the new loss after weights are updated
new_loss = np.mean((true_noise - predicted_noise_updated) ** 2)
print(f"Loss after update: {new_loss:.4f}")

Loss before update: 1.2785
Loss after update: 1.2662
